# SurvHive test

Let's compare the performance of three methods (Cox-based, tree-based and deep-learning-based) on the same dataset.
First the default parameters will be evaluated then a small random search on a grid will be performed.

In [1]:
import numpy
import sklearn
import survhive
import pandas

In [2]:
seed=202405

In [3]:
survhive.list_available_datasets()

('flchain', 'gbsg2', 'metabric', 'support')

Get the METABRIC example dataset

In [4]:
data_ds = survhive.get_data('metabric')
data_ds.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1903 entries, 0 to 1902
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   hormone_treatment  1903 non-null   float64
 1   radiotherapy       1903 non-null   float64
 2   chemoterapy        1903 non-null   float64
 3   ER-positive        1903 non-null   float64
 4   MKI67              1903 non-null   float64
 5   EGFR               1903 non-null   float64
 6   PGR                1903 non-null   float64
 7   ERBB2              1903 non-null   float64
 8   AOD                1903 non-null   float64
 9   time               1903 non-null   float64
 10  event              1903 non-null   int64  
dtypes: float64(10), int64(1)
memory usage: 178.4 KB


Look if any zero-time events are in dataset and remove them

In [5]:
if not data_ds.index_zero_times().empty :
    data_ds.discard_zero_times()

get data in scikit-survival format

In [6]:
X, y = data_ds.get_X_y()
X.shape, y.shape

((1903, 9), (1903,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
X_train, X_test, y_train, y_test = survhive.survival_train_test_split(X, y, rng_seed=seed)

In [9]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((1427, 9), (476, 9))

In [10]:
survhive.get_indicator(y).sum(), survhive.get_indicator(y_train).sum(), survhive.get_indicator(y_test).sum(),


(1103, 827, 276)

Set-up a dict to collect test results

In [11]:
vanilla_mods= {}
antolini_vanilla_test = {}

In [12]:
def model_class(_model):
    return type(_model).__name__

## setup a few vanilla models

Coxnet and GradientBoostingSA from sksurv, survtracesingle from survtrace 

In [13]:
vanilla_mods['CoxNet'] =survhive.CoxNet(rng_seed=seed)
vanilla_mods['GrBoostSA'] =survhive.GrBoostSA(rng_seed=seed)
vanilla_mods['SurvTraceSingle'] =survhive.SurvTraceSingle(rng_seed=seed)

In [14]:
for _ in vanilla_mods.keys():
    vanilla_mods[_].fit(X_train, y_train)
    antolini_vanilla_test[_] = vanilla_mods[_].score(X_test, y_test)

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/train_utils.py:208: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664817693763/work/torch/csrc/utils/python_arg_parser.cpp:1174.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


GPU not found! will use cpu for training!
[Train-0]: 34.17040836811066
[Val-0]: 1.6177984476089478
[Train-1]: 26.018836557865143
[Val-1]: 2.3166608810424805
EarlyStopping counter: 1 out of 5
[Train-2]: 22.699482768774033
[Val-2]: 1.2142997980117798
[Train-3]: 23.45330309867859
[Val-3]: 1.140151858329773
[Train-4]: 23.14783227443695
[Val-4]: 1.1459238529205322
EarlyStopping counter: 1 out of 5
[Train-5]: 22.748957574367523
[Val-5]: 1.140552282333374
EarlyStopping counter: 2 out of 5
[Train-6]: 22.698014616966248
[Val-6]: 1.132118582725525
[Train-7]: 23.213879942893982
[Val-7]: 1.1276575326919556
[Train-8]: 22.720844209194183
[Val-8]: 1.1209437847137451
[Train-9]: 22.828719437122345
[Val-9]: 1.1210485696792603
EarlyStopping counter: 1 out of 5
[Train-10]: 22.541648149490356
[Val-10]: 1.136405110359192
EarlyStopping counter: 2 out of 5
[Train-11]: 23.808623552322388
[Val-11]: 1.1252729892730713
EarlyStopping counter: 3 out of 5
[Train-12]: 22.302674889564514
[Val-12]: 1.1547764539718628
E

In [15]:
antolini_vanilla_test

{'CoxNet': 0.6230663234797078,
 'GrBoostSA': 0.6188414840236071,
 'SurvTraceSingle': 0.6330759431141617}

In [16]:
vanilla_df = pandas.DataFrame(data=antolini_vanilla_test.values(),
                              index=antolini_vanilla_test.keys(),
                             columns=['Defaults'])
vanilla_df

,Defaults
CoxNet,0.623066
GrBoostSA,0.618841
SurvTraceSingle,0.633076


# Parameter optimization

In [17]:
# max number of processes (CPU)
cpus=4
antolini_best_test = {}
from IPython.display import display
#display(pd1)
#display(pd2)

## Optimization loop

Now let's perform some random search on the model parameter space.

Each optimization step does a two repeats of five-fold cross validations. 
The score is the average on the internal cross validated tests.

In [18]:
#vanilla_mods['CoxNet'].get_parameter_grid()
for _model in vanilla_mods.keys():
    print("Optimizing", _model)
    # perform some points of random search on default search grid
    _opt_model, _opt_model_params, _opt_model_search = survhive.optimize(vanilla_mods[_model], 
                                                                         X_train, y_train, 
                                                                         mode='sklearn-random', tries=10, 
                                                                         n_jobs=cpus)
    print("Top-ranking model for", _model)
    print('N.B. mean_test_score is the mean of the cross-validation runs on the *training* data')
    display(survhive.get_model_top_ranking_df(_opt_model_search))
    print("Top-ten models for", _model)
    display(survhive.get_model_scores_df(_opt_model_search)[:10])
    antolini_best_test[_model]=_opt_model.score(X_test,y_test)

antolini_best_test

Optimizing CoxNet
Random search tries: 10
Top-ranking model for CoxNet
N.B. mean_test_score is the mean of the cross-validation runs on the *training* data


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
8,1,0.636914,0.018524,"{'l1_ratio': 0.01, 'alpha': 0.001}",0.00645,0.001438


Top-ten models for CoxNet


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
8,1,0.636914,0.018524,"{'l1_ratio': 0.01, 'alpha': 0.001}",0.00645,0.001438
2,2,0.636889,0.018477,"{'l1_ratio': 0.01, 'alpha': 0.005}",0.006083,0.000375
5,3,0.636744,0.017737,"{'l1_ratio': 0.1, 'alpha': 0.06}",0.007127,0.00191
9,4,0.636515,0.018635,"{'l1_ratio': 0.75, 'alpha': 0.005}",0.005932,0.000231
3,5,0.636469,0.018595,"{'l1_ratio': 0.9, 'alpha': 0.005}",0.006387,0.001362
4,6,0.636432,0.018501,"{'l1_ratio': 0.9, 'alpha': 0.008}",0.006022,0.000377
7,7,0.634473,0.017474,"{'l1_ratio': 0.25, 'alpha': 0.07}",0.007079,0.002507
0,8,0.622717,0.020459,"{'l1_ratio': 0.75, 'alpha': 0.07}",0.006202,0.000689
1,9,0.600229,0.019333,"{'l1_ratio': 0.25, 'alpha': 0.3}",0.00601,0.000492
6,10,0.539348,0.041217,"{'l1_ratio': 0.75, 'alpha': 0.3}",0.006794,0.000984


Optimizing GrBoostSA
Random search tries: 10
Top-ranking model for GrBoostSA
N.B. mean_test_score is the mean of the cross-validation runs on the *training* data


...

GPU not found! will use cpu for training!
[Train-0]: 34.544894218444824
[Val-0]: 1.6845825910568237
[Train-1]: 28.588575839996338
[Val-1]: 1.6858513355255127
EarlyStopping counter: 1 out of 5
[Train-2]: 23.54473513364792
[Val-2]: 1.1320862770080566
[Train-3]: 23.502846121788025
[Val-3]: 1.1438695192337036
EarlyStopping counter: 1 out of 5
[Train-4]: 23.14110654592514
[Val-4]: 1.1474288702011108
EarlyStopping counter: 2 out of 5
[Train-5]: 22.896730363368988
[Val-5]: 1.7695419788360596
EarlyStopping counter: 3 out of 5
[Train-6]: 23.050012826919556
[Val-6]: 1.244706392288208
EarlyStopping counter: 4 out of 5
[Train-7]: 23.385247111320496
[Val-7]: 1.1615666151046753
EarlyStopping counter: 5 out of 5
early stops at epoch 8
Top-ranking model for SurvTraceSingle
N.B. mean_test_score is the mean of the cross-validation runs on the *training* data


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
6,1,0.666928,0.018452,"{'num_hidden_layers': 2, 'num_attention_heads'...",2.489772,0.582178


Top-ten models for SurvTraceSingle


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
6,1,0.666928,0.018452,"{'num_hidden_layers': 2, 'num_attention_heads'...",2.489772,0.582178
1,2,0.666136,0.019834,"{'num_hidden_layers': 4, 'num_attention_heads'...",4.239228,0.921077
5,3,0.666067,0.019971,"{'num_hidden_layers': 3, 'num_attention_heads'...",3.49263,0.811693
3,4,0.665561,0.020394,"{'num_hidden_layers': 2, 'num_attention_heads'...",2.323099,0.697697
2,5,0.665101,0.021803,"{'num_hidden_layers': 2, 'num_attention_heads'...",2.590125,0.549475
9,6,0.664985,0.025519,"{'num_hidden_layers': 4, 'num_attention_heads'...",5.657013,1.030299
0,7,0.664394,0.019281,"{'num_hidden_layers': 4, 'num_attention_heads'...",3.677479,1.781468
8,8,0.664025,0.019849,"{'num_hidden_layers': 3, 'num_attention_heads'...",3.641658,1.028981
7,9,0.663168,0.022727,"{'num_hidden_layers': 2, 'num_attention_heads'...",3.202227,0.974203
4,10,0.662811,0.01685,"{'num_hidden_layers': 4, 'num_attention_heads'...",5.986478,1.731406


{'CoxNet': 0.6232743155760081,
 'GrBoostSA': 0.6154746119647453,
 'SurvTraceSingle': 0.6305150404284637}

# Final results

Performace of models expressed as an Antolini score on the test set.

In [21]:
optimized_df = pandas.DataFrame(data=antolini_best_test.values(),
                              index=antolini_best_test.keys(),
                             columns=['Somewhat optimized'])

In [22]:
pandas.concat([vanilla_df, optimized_df],axis=1)

,Defaults,Somewhat optimized
CoxNet,0.623066,0.623274
GrBoostSA,0.618841,0.615475
SurvTraceSingle,0.633076,0.630515


early stops at epoch 23
GPU not found! will use cpu for training!
[Train-0]: 29.03039515018463
[Val-0]: 1.486636757850647
[Train-1]: 23.682857871055603
[Val-1]: 1.188035488128662
[Train-2]: 19.081591248512268
[Val-2]: 1.1130684614181519
[Train-3]: 18.931862115859985
[Val-3]: 1.124733567237854
EarlyStopping counter: 1 out of 5
[Train-4]: 18.685747623443604
[Val-4]: 1.1045277118682861
[Train-5]: 18.697519421577454
[Val-5]: 1.131018042564392
EarlyStopping counter: 1 out of 5
[Train-6]: 18.525286614894867
[Val-6]: 1.1431227922439575
EarlyStopping counter: 2 out of 5
[Train-7]: 18.838000893592834
[Val-7]: 1.1244407892227173
EarlyStopping counter: 3 out of 5
[Train-8]: 17.988823533058167
[Val-8]: 1.1081739664077759
EarlyStopping counter: 4 out of 5
[Train-9]: 17.9964616894722
[Val-9]: 1.1044001579284668
[Train-10]: 18.168743312358856
[Val-10]: 1.089874505996704
[Train-11]: 18.23214703798294
[Val-11]: 1.1302227973937988
EarlyStopping counter: 1 out of 5
[Train-12]: 17.66786026954651
[Val-12]:

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")


[Val-11]: 1.0309025049209595
EarlyStopping counter: 4 out of 5
[Train-12]: 19.374826788902283
[Val-12]: 1.0329419374465942
EarlyStopping counter: 5 out of 5
early stops at epoch 13
GPU not found! will use cpu for training!
[Train-0]: 28.70712125301361
[Val-0]: 1.4970537424087524
[Train-1]: 24.689724922180176
[Val-1]: 1.4887659549713135
[Train-2]: 20.836036920547485
[Val-2]: 1.192684292793274
[Train-3]: 18.07558763027191
[Val-3]: 1.197619080543518
EarlyStopping counter: 1 out of 5
[Train-4]: 18.14040696620941
[Val-4]: 1.1739630699157715
[Train-5]: 17.230018585920334
[Val-5]: 1.2159456014633179
EarlyStopping counter: 1 out of 5
[Train-6]: 18.187370240688324
[Val-6]: 1.1946605443954468
EarlyStopping counter: 2 out of 5
[Train-7]: 17.65120106935501
[Val-7]: 1.186017394065857
EarlyStopping counter: 3 out of 5
[Train-8]: 18.04909384250641
[Val-8]: 1.215598225593567
EarlyStopping counter: 4 out of 5
[Train-9]: 18.372572243213654
[Val-9]: 1.1936378479003906
EarlyStopping counter: 5 out of 5
ea

/usr/local/ivan/Unito/survhive/survhive/sksurv_adapters.py:63: UserWarning: all coefficients are zero, consider decreasing alpha.
  self.model_ = self.model_.fit(X, y)
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/train_utils.py:208: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664817693763/work/torch/csrc/utils/python_arg_parser.cpp:1174.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
/usr/local/ivan/Unito/conda/envs/hive/li

[Train-7]: 17.87235915660858
[Val-7]: 1.1308684349060059
EarlyStopping counter: 4 out of 5
[Train-8]: 17.524035692214966
[Val-8]: 1.114086389541626
[Train-9]: 17.85876101255417
[Val-9]: 1.1076761484146118
[Train-10]: 17.745882153511047
[Val-10]: 1.1257798671722412
EarlyStopping counter: 1 out of 5
[Train-11]: 18.947903633117676
[Val-11]: 1.447114109992981
EarlyStopping counter: 2 out of 5
[Train-12]: 17.639041304588318
[Val-12]: 1.162989854812622
EarlyStopping counter: 3 out of 5
[Train-13]: 18.283275544643402
[Val-13]: 1.1615017652511597
EarlyStopping counter: 4 out of 5
[Train-14]: 17.74758380651474
[Val-14]: 1.138856291770935
EarlyStopping counter: 5 out of 5
early stops at epoch 15
GPU not found! will use cpu for training!
[Train-0]: 28.639700651168823
[Val-0]: 1.507087230682373
[Train-1]: 25.880115270614624
[Val-1]: 1.3571362495422363
[Train-2]: 21.750462889671326
[Val-2]: 1.2607383728027344
[Train-3]: 18.639665007591248
[Val-3]: 1.062756061553955
[Train-4]: 18.729476273059845
[Va

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Uni

EarlyStopping counter: 1 out of 5
[Train-4]: 18.73985779285431
[Val-4]: 1.094416856765747
[Train-5]: 18.76933604478836
[Val-5]: 1.0826191902160645
[Train-6]: 18.99302101135254
[Val-6]: 1.0797998905181885
[Train-7]: 17.787593722343445
[Val-7]: 1.1004045009613037
EarlyStopping counter: 1 out of 5
[Train-8]: 18.619486391544342
[Val-8]: 1.098008155822754
EarlyStopping counter: 2 out of 5
[Train-9]: 18.64366739988327
[Val-9]: 1.101132869720459
EarlyStopping counter: 3 out of 5
[Train-10]: 18.131380140781403
[Val-10]: 1.0862764120101929
EarlyStopping counter: 4 out of 5
[Train-11]: 17.943643629550934
[Val-11]: 1.0896843671798706
EarlyStopping counter: 5 out of 5
early stops at epoch 12
GPU not found! will use cpu for training!
[Train-0]: 28.93996500968933
[Val-0]: 1.4759511947631836
[Train-1]: 24.774123787879944
[Val-1]: 1.4107106924057007
[Train-2]: 20.546071410179138
[Val-2]: 1.1917946338653564
[Train-3]: 18.909303903579712
[Val-3]: 1.3729262351989746
EarlyStopping counter: 1 out of 5
[Tra

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/usr/local/ivan/Uni